# `libcusmm` explore training data

This notebook allows you to explore the training data collected from autotuning before proceeding to training.

### Import libraries

In [ ]:
import re, sys, os, json
import numpy as np
import pandas as pd

### Path to autotuning data

Provide the path to the autotuning data:
- You can use the bash cell below to navigate your filetree. 
- Then, copy what you've replaced `AUTOTUNING_DATA_PATH` with in the Python variable `autotuning_data_path` below

In [ ]:
%%bash
ls -ad AUTOTUNING_DATA_PATH/tune_dataset/tune_*x*x*/

In [ ]:
%%bash
ls -ad ~/scratch/tune_dataset/tune_*x*x*/ | wc -l 

In [ ]:
autotuning_data_path = '/users/alicej/scratch/tune_dataset'  # may not recognize '~', provide absolute complete paths
assert os.path.exists(autotuning_data_path)
assert len(os.listdir(autotuning_data_path)) > 0

### Set options

Set the following options appropriately:

In [ ]:
to_read = 100       # How many / which data folders to read. Options: 
                    # - 'all': reads from all available data folders. 
                    #   Beware, this might result in memory errors if large amounts of data are made available
                    # - a number: reads this number of data folders (e.g. 100)
                    # - a regex: reads the data folders with matching regex (e.g. tune_4x*x*)

In [ ]:
kernel_folder_pattern = re.compile('tune_(\d+)x(\d+)x(\d+)$')

# Get the list of folders to read
if to_read == 'all':
    folders_to_read = [os.path.join(autotuning_data_path, f) for f in os.listdir(autotuning_data_path) 
                       if kernel_folder_pattern.match(f) is not None]
elif isinstance(to_read, int):
    folders_to_read = [os.path.join(autotuning_data_path, f) for f in os.listdir(autotuning_data_path) 
                       if kernel_folder_pattern.match(f) is not None]
    folders_to_read = folders_to_read[:to_read]
elif isinstance(to_read, str):
    to_read = re.compile(to_read)
    folders_to_read = [os.path.join(autotuning_data_path, f) for f in os.listdir(autotuning_data_path) 
                       if to_read.match(f) is not None]    
else: 
    assert False, "Cannot recognize option: " + to_read

num_folders_to_read = len(folders_to_read)
assert num_folders_to_read > 0
print("Data folders to be read from (total: {:,})\n".format(num_folders_to_read))
[print(f) for f in folders_to_read]

In [ ]:
algorithm = 'all'   # algorithms to explore. Options: all, tiny, small, medium
                    #-------------------------------------------------------------------
algo_to_read = [algorithm] if algorithm != 'all' else ['tiny', 'small', 'medium', 'largeDB1', 'largeDB2']
print("Algorithm(s) to explore:")
[print(a) for a in algo_to_read]

In [ ]:
thresh = 300000     # do not perform very long operations on row counts above this threshold
                    #-------------------------------------------------------------------

### Read training data

In [ ]:
dataframe_list = list()
kernel_folder_pattern = re.compile('tune_(\d+x\d+x\d+)$')

In [ ]:
for i, kernel_folder in enumerate(folders_to_read):
    print('\nfrom {}, read                                  ({}/{:,})'.format(kernel_folder, i+1, num_folders_to_read))
        
    for name_algo in algo_to_read: 

        mnk_string = kernel_folder_pattern.search(kernel_folder).groups()[0]
        raw_file_base = 'raw_training_data_' + mnk_string + '_' + name_algo + '.csv'
        raw_file = os.path.join(kernel_folder, raw_file_base)
        derived_file_base = 'training_data_' + mnk_string + '_' + name_algo + '.csv'
        derived_file = os.path.join(kernel_folder, derived_file_base) 

        if os.path.exists(raw_file) and os.path.exists(derived_file):

            # Read raw parameters file
            raw = pd.read_csv(raw_file, index_col=0)
            raw['algorithm'] = np.array([name_algo]*len(raw.index.values))
            print('\t{:50} number of lines: {:>8,}'.format(raw_file_base, len(raw.index.values)))

            # Read derived parameters file
            derived = pd.read_csv(derived_file, index_col=0)
            print('\t{:50} number of lines: {:>8,}'.format(derived_file_base, len(raw.index.values)))
            dataframe_list.append(pd.concat([raw, derived], axis=1))

        else: 
            
            if not os.path.exists(raw_file):
                print('\t...{:50} no file'.format(raw_file_base))
            if not os.path.exists(derived_file):
                print('\t...{:50} no file'.format(derived_file_base))
            
print('Read all csv files, merging dataframes')
data = pd.concat(dataframe_list, ignore_index=True)
data.dropna(axis=1, how='all', inplace=True)

### Data head

In [ ]:
page_width = 5 # columns per output line
for i in range(0, len(data.columns.values), page_width):
    display(data.iloc[:,i:i+page_width].head())

### Data description

In [ ]:
import sys
print('Data size        :', sys.getsizeof(data)/10**6, 'MB')
print('Number of columns:', len(data.columns.values))
print('Number of rows   : {:,}'.format(len(data.index.values)))

In [ ]:
for i in range(0, len(data.columns.values), page_width):
    display(data.iloc[:,i:i+page_width].describe())

### Columns

In [ ]:
print('Number of columns:', len(data.columns), '\nNumber of rows:', len(data.index.values), '\n')
for col in data.columns: 
    print('{:<40} {}'.format(col, data[col].dtype))

In [ ]:
# Feature categories
mnk = ['m', 'n', 'k']
kernel_pars = ['algorithm', 'threads_per_blk', 'grouping', 'minblocks',
               'tile_m', 'tile_n', 'w', 'v', 'nbytes_smem', 'nbytes_cmem', 'regs_per_thread']
kernel_pars = list(set(kernel_pars) & set(data.columns.values))
perf =  ['perf (Gflop/s)', 'perf_squared', 'perf_scaled', 'perf_scaled_by_algo']
common = ['Gflops', 'mxnxk', 'size_a', 'size_b', 'size_c', 'nblks', 
          'warps_per_blk', 'nwarps', 'sm_desired', 'nthreads', 'ru_param_stack_unroll_factor']

### Sample row

In [ ]:
import random
other_columns = list(set(data.columns.values) - set(kernel_pars + mnk + perf + common))
irow = 0#random.sample(range(len(data.index.values)), 1)[0]
row = data.iloc[irow,:]
print(row[mnk], '\n')
print(row[kernel_pars], '\n')
print(row[perf], '\n')
print(row[common], '\n')
print(row[other_columns], '\n')

### Features

Features in the left-most column correspond to "raw" parameters
* **green** kernel parameters 
* **grey** CUDA card properties (taken from CUDA documentation) 
* **pink** autotuning parameters (taken from DBCSR codebase) 

Other features correspond to derived parameters, computed from the "raw" parameters
* **yellow** matrix sizes
* **light grey** launch parameters
* **blue** and **purple** estimations of resource usages

![parameters dependency graph](libcusmm_parameters_and_memory.png)

In [ ]:
data_profiling = data
if len(data.index.values) > thresh:  # if it is a very large dataframe, perform op on subsampled rows 
    sampled_rows = random.sample(data_profiling.index.values.tolist(), thresh)
    data_profiling = data.iloc[sampled_rows,:]

import pandas_profiling 
pandas_profiling.ProfileReport(data_profiling)

### Data visualization

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.semilogx(data['mxnxk'], data['perf (Gflop/s)'], '.', markersize=1)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance [Gflops]')

In [ ]:
plt.semilogx(data['mxnxk'], data['perf_squared'], '.', markersize=1)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance squared [Gflops]^2')

### Data visualization (scaled performance)

In [ ]:
plt.plot(data['mxnxk'], data['perf_scaled'], '.', markersize=1)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance scaled (overall)')

In [ ]:
algos = np.unique(data['algorithm'])
for algo in algos:
    algo_data = data[data['algorithm'] == algo]
    plt.figure()
    plt.semilogx(algo_data['mxnxk'], algo_data['perf_scaled_by_algo'], '.', markersize=1)
    plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
    plt.ylabel('Performance scaled (by algorithm)')
    plt.title(algo)
    plt.show()

### Performance profile

In [ ]:
# Choose (m, n, k) triplet
m_plot, n_plot, k_plot = (4, 4, 4)

In [ ]:
data_mnk = data[data['m'] == m_plot][ 
                data['n'] == n_plot][ 
                data['k'] == k_plot]
data_mnk.sort_values(by='perf (Gflop/s)', ascending=True, inplace=True)
plt.plot(data_mnk['perf (Gflop/s)'].values)
plt.xlabel('parameter set')
plt.ylabel('perf (Gflop/s)')
plt.title('Performance profile for kernel ' + str(m_plot) + 'x'+ str(n_plot) + 'x'+ str(k_plot))

In [ ]:
# Histograms with Bokeh
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
output_notebook()

# Create histogram
num_bins = 100 
hist, edges = np.histogram(data_mnk['perf (Gflop/s)'], bins=num_bins)
df_hist = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
source = ColumnDataSource(df_hist)

# Create tool 
hover = HoverTool(tooltips=[('# occurences', '@hist'), ('low', '@left'), ('high', '@right')])

# Create the figure
p = figure(plot_width=800, plot_height=800, title="Performance histogram",
           toolbar_location=None, tools="")
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Performance (GFlop/s)"
p.xaxis.major_label_orientation = 1.2
p.yaxis.axis_label = "# occurrences"
p.quad(source=source, bottom=0, top='hist', left='left', right='right', fill_color='blue')
p.add_tools(hover)
show(p)


In [ ]:
# Histograms with Bokeh
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
output_notebook()

# Create histogram
num_bins = 100 
hist, edges = np.histogram(data_mnk['perf_scaled'], bins=num_bins)
df_hist = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
source = ColumnDataSource(df_hist)

# Create tool 
hover = HoverTool(tooltips=[('# occurences', '@hist'), ('low', '@left'), ('high', '@right')])

# Create the figure
p = figure(plot_width=800, plot_height=800, title="Performance histogram",
           toolbar_location=None, tools="")
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Performance scaled"
p.xaxis.major_label_orientation = 1.2
p.yaxis.axis_label = "# occurrences"
p.quad(source=source, bottom=0, top='hist', left='left', right='right', fill_color='blue')
p.add_tools(hover)
show(p)

In [ ]:
# Top slices of perf. distribution
pars_autotuning_top = {
    5: list(), 
    2: list(), 
    1: list(), 
    0.5: list()
}
max_perf = float(data_mnk['perf (Gflop/s)'].max())
max_perf_idx = data_mnk['perf (Gflop/s)'].idxmax()
max_perf_row = data_mnk.loc[max_perf_idx]
max_perf_cond = max_perf_row[mnk + kernel_pars + ['perf (Gflop/s)']]

print('Maximally performing parameter set:')
display(max_perf_cond)
for perc in pars_autotuning_top.keys():
    lim = max_perf - max_perf*perc/100
    blob = data_mnk.loc[data_mnk['perf (Gflop/s)'] >= lim]
    print('\ntop', perc, '%')
    display(blob[kernel_pars + ['perf (Gflop/s)']].describe())
    pars_autotuning_top[perc].append(blob)

### Pair plot 

In [ ]:
data_pairplot = data
if len(data.index.values) > thresh:  # if it is a very large dataframe, perform op on subsampled rows 
    sampled_rows = random.sample(data_pairplot.index.values.tolist(), thresh)
    data_pairplot = data.iloc[sampled_rows,:]

sns.pairplot(data_pairplot[mnk + kernel_pars + perf].dropna())